In [15]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import set_config; set_config(display='diagram')
from cryptobot.yahoo_market_data import get_yahoo_data


BASE_DATA_PATH = "../../CryptoBotPrueba/ETHUSDT-1m-2022-05.csv"
COLUMN_NAMES = ["open_time","open","high","low", "close",
                         "volume", "close_time" ,"quote_asset_volume",
                         "number_of_trades", "taker_buy_base_asset_volume",
                         "taker_buy_quote_asset_volume", "ignore"]

def get_data_without_headers(path, columns):
    df = pd.read_csv(path,
                 header=None, 
                 names= columns)
    df.open_time = df.open_time.apply(lambda x: datetime.utcfromtimestamp(x/1000))
    df.close_time = df.close_time.apply(lambda x: datetime.utcfromtimestamp(x/1000))
    df["close_time_str"] = df["close_time"].apply(lambda x: x.strftime("%Y-%m-%d"))
    return df

df = get_data_without_headers(BASE_DATA_PATH,COLUMN_NAMES)
df.head(3)

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,close_time_str
0,2022-05-01 00:00:00,2726.67,2729.36,2725.86,2728.04,628.9811,2022-05-01 00:00:59.999,1.715749e+06,700,348.2220,949877.534398,0,2022-05-01
1,2022-05-01 00:01:00,2728.05,2728.59,2726.51,2727.20,368.7584,2022-05-01 00:01:59.999,1.005854e+06,484,279.6466,762819.463711,0,2022-05-01
2,2022-05-01 00:02:00,2727.21,2731.74,2727.20,2731.67,563.1505,2022-05-01 00:02:59.999,1.537359e+06,705,364.8842,995979.439779,0,2022-05-01


In [16]:
FEAR_GREED_PATH = "../../CryptoBotPrueba/fear_greed_index.csv"

def get_data_with_headers(path):
    fg = pd.read_csv(path)
    fg["timestamp"] = fg.timestamp.apply(lambda x: datetime.utcfromtimestamp(int(x)))
    fg["close_time_str"] = fg["timestamp"].apply(lambda x: x.strftime("%Y-%m-%d"))
    return fg

fg = get_data_with_headers(FEAR_GREED_PATH)
fg.head(3)

,Unnamed: 0,value,value_classification,timestamp,close_time_str
0,0,13,Extreme Fear,2022-06-06,2022-06-06
1,1,10,Extreme Fear,2022-06-05,2022-06-05
2,2,14,Extreme Fear,2022-06-04,2022-06-04


In [17]:
def merge_df_fg(df,fg):
    df_fg = df.merge(fg, on = "close_time_str" )
    return df_fg

df_fg =  merge_df_fg(df,fg)
df_fg.head(3)

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,close_time_str,Unnamed: 0,value,value_classification,timestamp
0,2022-05-01 00:00:00,2726.67,2729.36,2725.86,2728.04,628.9811,2022-05-01 00:00:59.999,1.715749e+06,700,348.2220,949877.534398,0,2022-05-01,36,22,Extreme Fear,2022-05-01
1,2022-05-01 00:01:00,2728.05,2728.59,2726.51,2727.20,368.7584,2022-05-01 00:01:59.999,1.005854e+06,484,279.6466,762819.463711,0,2022-05-01,36,22,Extreme Fear,2022-05-01
2,2022-05-01 00:02:00,2727.21,2731.74,2727.20,2731.67,563.1505,2022-05-01 00:02:59.999,1.537359e+06,705,364.8842,995979.439779,0,2022-05-01,36,22,Extreme Fear,2022-05-01


In [18]:
""" Loop for importing datasets with various financial indices   
    
    '^IXIC' - Nasdaq compound index
    '^GSPC' - S&P 500 index
    'GC=F' - Gold
    '^DJI' - Dow Jones Industrial Average
    '^TNX' = S&P 500 US T-bills (10 year)
    'DX-Y.NYB'= US Dollar/USDX - Index - Cash

"""
# Index'^IXIC' with import format error! Add to INDEX_LIST when solved
# Index '^TNX' with "Volume" 0 value in all rows
# TODO: NAN values when left merging!

INDEX_LIST = ['^IXIC','^GSPC','GC=F','^DJI', '^TNX','DX-Y.NYB']
START_DATE = "2020-05-05"
END_DATE = "2022-06-07"

from cryptobot.yahoo_market_data import get_yahoo_data

for i in INDEX_LIST:
    yd = get_yahoo_data(i, START_DATE, END_DATE)
    yd[f'{i}_Date'] = yd[f'{i}_Date'].apply(lambda x: x.strftime("%Y-%m-%d"))
    yd.rename(columns = {f'{i}_Date':"close_time_str"}, inplace = True)
    df_fg = df_fg.merge(yd, on = "close_time_str", how="left" )

df_fg

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,^TNX_Open,^TNX_High,^TNX_Low,^TNX_Close,^TNX_timestamp,DX-Y.NYB_Open,DX-Y.NYB_High,DX-Y.NYB_Low,DX-Y.NYB_Close,DX-Y.NYB_timestamp
0,2022-05-01 00:00:00,2726.67,2729.36,2725.86,2728.04,628.9811,2022-05-01 00:00:59.999,1.715749e+06,700,348.2220,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-05-01 00:01:00,2728.05,2728.59,2726.51,2727.20,368.7584,2022-05-01 00:01:59.999,1.005854e+06,484,279.6466,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-05-01 00:02:00,2727.21,2731.74,2727.20,2731.67,563.1505,2022-05-01 00:02:59.999,1.537359e+06,705,364.8842,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-05-01 00:03:00,2731.67,2733.17,2727.18,2733.16,3325.9394,2022-05-01 00:03:59.999,9.073275e+06,1266,2400.7013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-05-01 00:04:00,2733.16,2733.17,2730.84,2733.03,278.3504,2022-05-01 00:04:59.999,7.605418e+05,496,137.1773,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44635,2022-05-31 23:55:00,1943.71,1946.16,1940.00,1940.66,1385.0172,2022-05-31 23:55:59.999,2.692265e+06,981,411.7625,...,2.83,2.877,2.828,2.844,1.653955e+09,101.440002,102.169998,101.410004,101.75,1.653955e+09
44636,2022-05-31 23:56:00,1940.66,1940.94,1938.04,1939.25,914.3149,2022-05-31 23:56:59.999,1.773253e+06,927,361.3201,...,2.83,2.877,2.828,2.844,1.653955e+09,101.440002,102.169998,101.410004,101.75,1.653955e+09
44637,2022-05-31 23:57:00,1939.25,1941.79,1938.39,1940.87,627.5494,2022-05-31 23:57:59.999,1.217479e+06,499,280.2652,...,2.83,2.877,2.828,2.844,1.653955e+09,101.440002,102.169998,101.410004,101.75,1.653955e+09
44638,2022-05-31 23:58:00,1940.87,1942.00,1939.67,1939.68,562.3038,2022-05-31 23:58:59.999,1.091493e+06,425,175.9596,...,2.83,2.877,2.828,2.844,1.653955e+09,101.440002,102.169998,101.410004,101.75,1.653955e+09


In [19]:
# We could use the fillna method on the DataFrame and specify the method as ffill (forward fill):

# df.fillna(method='ffill') 
# This method propagate[s] last valid observation forward to next valid

#df.fillna(method="bfill")
# This method propagate[s] last valid observation backward to last valid

df_fg=df_fg.fillna(method="bfill").fillna(method="ffill")

In [20]:
df_fg.isna().sum()    

open_time                       0
open                            0
high                            0
low                             0
close                           0
volume                          0
close_time                      0
quote_asset_volume              0
number_of_trades                0
taker_buy_base_asset_volume     0
taker_buy_quote_asset_volume    0
ignore                          0
close_time_str                  0
Unnamed: 0                      0
value                           0
value_classification            0
timestamp                       0
^IXIC_Open                      0
^IXIC_High                      0
^IXIC_Low                       0
^IXIC_Close                     0
^IXIC_timestamp                 0
^GSPC_Open                      0
^GSPC_High                      0
^GSPC_Low                       0
^GSPC_Close                     0
^GSPC_timestamp                 0
GC=F_Open                       0
GC=F_High                       0
GC=F_Low      

In [21]:
def define_target(df):
    df["target"] = (df.open - df.close).apply(lambda x: 0 if x <=0 else 1)
    return df

df = define_target(df_fg)
df_fg.head(3)

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,^TNX_High,^TNX_Low,^TNX_Close,^TNX_timestamp,DX-Y.NYB_Open,DX-Y.NYB_High,DX-Y.NYB_Low,DX-Y.NYB_Close,DX-Y.NYB_timestamp,target
0,2022-05-01 00:00:00,2726.67,2729.36,2725.86,2728.04,628.9811,2022-05-01 00:00:59.999,1.715749e+06,700,348.2220,...,3.002,2.907,2.996,1.651450e+09,103.209999,103.75,103.110001,103.739998,1.651450e+09,0
1,2022-05-01 00:01:00,2728.05,2728.59,2726.51,2727.20,368.7584,2022-05-01 00:01:59.999,1.005854e+06,484,279.6466,...,3.002,2.907,2.996,1.651450e+09,103.209999,103.75,103.110001,103.739998,1.651450e+09,1
2,2022-05-01 00:02:00,2727.21,2731.74,2727.20,2731.67,563.1505,2022-05-01 00:02:59.999,1.537359e+06,705,364.8842,...,3.002,2.907,2.996,1.651450e+09,103.209999,103.75,103.110001,103.739998,1.651450e+09,0


In [22]:
# from sklearn.pipeline import FeatureUnion
# from sklearn.preprocessing import FunctionTransformer

# Create a custom transformer that multiplies two columns
# target = FunctionTransformer((df.open - df.close).apply(lambda x: 0 if x <=0 else 1))


# union = FeatureUnion([
#     ("preprocess", preprocessor), # columns 0-11
#     ("target", target), # new column "target"

# ])
# union

In [23]:
X=df_fg.drop(columns="target")
y=df_fg["target"]

In [30]:
from sklearn.base import TransformerMixin, BaseEstimator #gives fit_transform method for free
class MyTransformer(TransformerMixin, BaseEstimator):
    def __init__(self):
        self.encoder = LabelEncoder()
    def fit(self, x, y=None):
        self.encoder.fit(x)
        return self
    def transform(self, x, y=None):
        return self.encoder.transform(x)


In [33]:
# Select num columns to scale: 
from sklearn.compose import make_column_selector

NUM_COL_LIST = X.select_dtypes(include=["float64","int64"]).columns.values.tolist()
CAT_COL_LIST = ["value_classification"]

# Impute then Scale for numerical variables: 
num_transformer = RobustScaler()

# Encode categorical variables

# Try Ordinal Encoder!!!!

cat_transformer = OneHotEncoder()

# Paralellize "num_transformer" and "One hot encoder"
preprocessor = ColumnTransformer([
                                ('num_tr', num_transformer, NUM_COL_LIST),
                                ('cat_tr', cat_transformer, CAT_COL_LIST)
                                ],
                                remainder='passthrough')

preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('num_tr', RobustScaler(),
                                 ['open', 'high', 'low', 'close', 'volume',
                                  'quote_asset_volume', 'number_of_trades',
                                  'taker_buy_base_asset_volume',
                                  'taker_buy_quote_asset_volume', 'ignore',
                                  'Unnamed: 0', 'value', '^IXIC_Open',
                                  '^IXIC_High', '^IXIC_Low', '^IXIC_Close',
                                  '^IXIC_timestamp', '^GSPC_Open', '^GSPC_High',
                                  '^GSPC_Low', '^GSPC_Close', '^GSPC_timestamp',
                                  'GC=F_Open', 'GC=F_High', 'GC=F_Low',
                                  'GC=F_Close', 'GC=F_timestamp', '^DJI_Open',
                                  '^DJI_High', '^DJI_Low', ...]),
                                ('cat_tr', OneHotEncoder(),
                                 ['value_classification'])])

In [34]:
pd.DataFrame(preprocessor.fit_transform(X), columns=preprocessor.get_feature_names_out())

,num_tr__open,num_tr__high,num_tr__low,num_tr__close,num_tr__volume,num_tr__quote_asset_volume,num_tr__number_of_trades,num_tr__taker_buy_base_asset_volume,num_tr__taker_buy_quote_asset_volume,num_tr__ignore,...,num_tr__DX-Y.NYB_High,num_tr__DX-Y.NYB_Low,num_tr__DX-Y.NYB_Close,num_tr__DX-Y.NYB_timestamp,cat_tr__value_classification_Extreme Fear,cat_tr__value_classification_Fear,remainder__open_time,remainder__close_time,remainder__close_time_str,remainder__timestamp
0,1.206339,1.207481,1.208926,1.208827,0.651466,0.999095,0.531621,0.782311,1.163483,0.0,...,0.0,0.048486,0.154284,-0.933333,1.0,0.0,2022-05-01 00:00:00,2022-05-01 00:00:59.999,2022-05-01,2022-05-01
1,1.208751,1.206136,1.210063,1.207358,0.114019,0.313959,0.104743,0.505949,0.810298,0.0,...,0.0,0.048486,0.154284,-0.933333,1.0,0.0,2022-05-01 00:01:00,2022-05-01 00:01:59.999,2022-05-01,2022-05-01
2,1.207283,1.211638,1.21127,1.215173,0.515504,0.826927,0.541502,0.849461,1.250528,0.0,...,0.0,0.048486,0.154284,-0.933333,1.0,0.0,2022-05-01 00:02:00,2022-05-01 00:02:59.999,2022-05-01,2022-05-01
3,1.215079,1.214136,1.211235,1.217777,6.221581,8.100016,1.650198,9.053918,11.733478,0.0,...,0.0,0.048486,0.154284,-0.933333,1.0,0.0,2022-05-01 00:03:00,2022-05-01 00:03:59.999,2022-05-01,2022-05-01
4,1.217684,1.214136,1.217639,1.21755,-0.072703,0.077202,0.128458,-0.068211,0.077656,0.0,...,0.0,0.048486,0.154284,-0.933333,1.0,0.0,2022-05-01 00:04:00,2022-05-01 00:04:59.999,2022-05-01,2022-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44635,-0.162365,-0.1605,-0.165941,-0.16767,2.212932,1.941553,1.086957,1.038383,0.881036,0.0,...,-0.908044,-0.981814,-0.982858,1.0,1.0,0.0,2022-05-31 23:55:00,2022-05-31 23:55:59.999,2022-05-31,2022-05-31
44636,-0.167697,-0.169618,-0.16937,-0.170135,1.240775,1.054594,0.980237,0.835098,0.693035,0.0,...,-0.908044,-0.981814,-0.982858,1.0,1.0,0.0,2022-05-31 23:56:00,2022-05-31 23:56:59.999,2022-05-31,2022-05-31
44637,-0.170161,-0.168133,-0.168757,-0.167303,0.648509,0.518203,0.134387,0.508442,0.39648,0.0,...,-0.908044,-0.981814,-0.982858,1.0,1.0,0.0,2022-05-31 23:57:00,2022-05-31 23:57:59.999,2022-05-31,2022-05-31
44638,-0.167329,-0.167766,-0.166518,-0.169383,0.513755,0.396611,-0.011858,0.088084,0.014874,0.0,...,-0.908044,-0.981814,-0.982858,1.0,1.0,0.0,2022-05-31 23:58:00,2022-05-31 23:58:59.999,2022-05-31,2022-05-31


In [ ]:
# def split_data(df):
#    X=df[["open_time", "open", "high", "low", "close",
#         "volume", "close_time", "quote_asset_volume",
#         "number_of_trades", "taker_buy_base_asset_volume", 
#         "taker_buy_quote_asset_volume", "value","value_classification"]]
#    y=pd.DataFrame(df["target"])
#    return (X,y)

# X,y = split_data(df_fg)


In [ ]:
# Scaling numerical values

NUM_COL_LIST = df_fg.select_dtypes(include=["float64","int64"]).columns

def num_scaler(df_fg, columns):
    
    r_scaler = RobustScaler() # Instanciate Robust Scaler
    r_scaler.fit(df_fg[columns]) # Fit scaler to feature
    df_fg[NUM_COL_LIST] = r_scaler.transform(df_fg[NUM_COL_LIST]) #Scale
    return df_fg

df_fg = num_scaler(df_fg, NUM_COL_LIST)
df_fg.head(3)

In [ ]:
# Encoding categorical values
# Build an ordinal encoder

def value_class_encoder(df_fg):
    le = LabelEncoder()
    le.fit(df_fg['value_classification'])
    df_fg['value_classification'] = le.transform(df_fg['value_classification'])
    return df_fg

df_fg = value_class_encoder(df_fg)
df_fg['value_classification'].unique()

